In [ ]:
from google.colab import drive
root = '/content/drive/'
drive.mount(root)

Mounted at /content/drive/


In [ ]:
# Making Directory
import os 
from os.path import join

tracking = "/content/tracking"   # a custom path. you can change if you want to
TRACKING_PATH = join(root,tracking)
!mkdir "{TRACKING_PATH}"

In [ ]:
#download personal dataset
!unzip /content/drive/MyDrive/inference_noImages.zip

Archive:  /content/drive/MyDrive/inference_noImages.zip
replace inference_noImages/inner/pre_inference_annotations.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: mv: cannot stat '/content/inference_noImages/content/rgb': No such file or directory


In [ ]:
!mv /content/inference_noImages /content/tracking

In [ ]:
import sys

tracking_data = join(TRACKING_PATH,'inner')
sys.path.append(tracking_data)

In [ ]:
#copy the json annotations in order to perform tracking on bounding boxes
!cp /content/tracking/inference_noImages/inner/rgb/pretracking_rgb.json /content/tracking/annotations_new.json

In [ ]:
# Example: Original picture BEFORE detection
import matplotlib.pylab as plt
import pylab
pylab.rcParams['figure.figsize'] = (11.25, 20.0)
import cv2

list_data = os.listdir(tracking_data)  
list_data.sort()

img_ex_path = tracking_data + "/" + list_data[0] #load first image
img_ex_origin = cv2.imread(img_ex_path)
img_ex = cv2.cvtColor(img_ex_origin, cv2.COLOR_BGR2RGB)

plt.imshow(img_ex)
plt.axis('off')
plt.show()


#START COUNTING WITH SORT USING ALL THE JSON PRODUCED BY MODELS

In [ ]:
#load annotations
import json
import collections

#insert json we want to use
jsonpath = join(TRACKING_PATH,'/content/tracking/inference_noImages/outer/rgbd/tracking_rgbd_Centered.json')

with open(jsonpath) as data_file:    
   data = json.load(data_file)
odata = collections.OrderedDict(sorted(data.items()))

In [ ]:
key = 'rgb1630.png'
data = odata['rgb1630.png']

In [ ]:
data[0]

{'image_id': 1630,
 'category_id': 0,
 'bbox': [452.9709167480469,
  768.1900634765625,
  41.75543212890625,
  72.3905029296875],
 'score': 0.9997641444206238,
 'segmentation': {'size': [1280, 720],
  'counts': 'aPga04jW19H5L4L2N1O1O2N>B2N1ciNdNSV1^1jiNgNPV1\\1niNfNQV1e16JE\\NZjNd1fU1\\NYjNe1fU1]NXjNd1hU1;O0000FUjN^NlU1a1WjN]NiU1`1[jN^NfU1]1a0N100N3O1O2O1O1O1POZiNd0gV1YO]iNd0RW1[OhhN<`W1J7M101O002LS_h8'},
 'instance_id': 5813}

In [ ]:
img_path = '/content/tracking/rgb/rgb2900.png'

img_ex_origin = cv2.imread(img_path)
img_ex = cv2.cvtColor(img_ex_origin, cv2.COLOR_BGR2RGB)

det_result = data 

for info in det_result:
    bbox = info['bbox']
    labels = info['label']
    scores = info['scores']
    templist = bbox+[float(scores)]

    for arr in bbox:
      x = int(bbox[0])
      y = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      cv2.rectangle(img_ex, (x, y), (x2, y2), (0, 255, 255), 4)

plt.imshow(img_ex)
plt.axis('off')
plt.show()

##now perform tracking

In [ ]:
#unzip sort folder 
!unzip /content/drive/MyDrive/sort.zip
#install SORT dependencies
!python -m pip install -r "/content/sort/requirements.txt"
#resolve conflicts
!pip uninstall imgaug
!pip install imgaug==0.2.6

In [ ]:
!mv /content/sort /content/tracking

In [ ]:
sort = join(TRACKING_PATH,'sort/')
sys.path.append(sort)

In [ ]:
#rerun this cell twice, don't know why
from pprint import pprint
from sort import *

In [ ]:
img_path = tracking_data    # img root path

# Making new directory for saving results
save_path = join(TRACKING_PATH,'save')
!mkdir "{save_path}"

In [ ]:
#perform tracking on custom dataset, then save the image with the id found by the algorithm for each frame
KalmanBoxTracker.count=0
mot_tracker_def = Sort()#max_age=1,min_hits=3, iou_threshold=0.3)

with open(jsonpath) as data_file:    
   data = json.load(data_file)
odata = collections.OrderedDict(sorted(data.items()))

##COUNTING
ids_dict = {}

for key in odata.keys():
    print(key)
    arrlist = []
    #image_name = key.split("/")[3]
    #det_img = cv2.imread(os.path.join(img_path, image_name))
    #overlay = det_img.copy()
    det_result = data[key] 
    
    for info in det_result:
        bbox = info['bbox']
        #print(bbox)
        xmin = bbox[0]
        ymin = bbox[1]
        xmax = bbox[0] + bbox[2]
        ymax = bbox[1] + bbox[3]
        new_bbox = [xmin, ymin, xmax, ymax]
        #print(new_bbox)
        
        #labels = info['category_id']
        scores = info['score']
        templist = new_bbox+[float(scores)]
        
        #if labels == 0:
        arrlist.append(templist)

           
    track_bbs_ids = mot_tracker_def.update(np.array(arrlist))
    
    ###################################################################################
    #COUNT FROM TRACKING:
    #here insert smth that keeps track of the count of different IDs in the frames and count the ones which appear after a certain treshold
    #e.g ID 1 appear in 40 frames, ID 68 appear in 5 frames, count IDs appearing over 30 frames
    #NB they should be smth like -64-
    for elem in track_bbs_ids:
        id_bbox = int(elem[-1])
        #print(elem)
        if id_bbox in ids_dict:
            value = ids_dict[id_bbox]
            ids_dict[id_bbox] = value + 1
        else:
            ids_dict[id_bbox] = 1
    #################################################################################

    #mot_imgid = key.replace('.png','')
    #img_name = mot_imgid.split("/")[3]
    #newname = save_path + '/' + img_name + '_mot.png'
    #print(mot_imgid)
    
    #for j in range(track_bbs_ids.shape[0]):  
    #    ele = track_bbs_ids[j, :]
    #    x = int(ele[0])
    #    y = int(ele[1])
    #    x2 = int(ele[2])
    #    y2 = int(ele[3])
    #    track_label = str(int(ele[4])) 
    #    cv2.rectangle(det_img, (x, y), (x2, y2), (0, 255, 255), 4)
    #    cv2.putText(det_img, '#'+track_label, (x+5, y-10), 0,0.6,(0,255,255),thickness=2)
        
    #cv2.imwrite(newname,det_img)


rgb3620.png
rgb3621.png
rgb3622.png
rgb3623.png
rgb3624.png
rgb3625.png
rgb3626.png
rgb3627.png
rgb3628.png
rgb3629.png
rgb3630.png
rgb3631.png
rgb3632.png
rgb3633.png
rgb3634.png
rgb3635.png
rgb3636.png
rgb3637.png
rgb3638.png
rgb3639.png
rgb3640.png
rgb3641.png
rgb3642.png
rgb3643.png
rgb3644.png
rgb3645.png
rgb3646.png
rgb3647.png
rgb3648.png
rgb3649.png
rgb3650.png
rgb3651.png
rgb3652.png
rgb3653.png
rgb3654.png
rgb3655.png
rgb3656.png
rgb3657.png
rgb3658.png
rgb3659.png
rgb3660.png
rgb3661.png
rgb3662.png
rgb3663.png
rgb3664.png
rgb3665.png
rgb3666.png
rgb3667.png
rgb3668.png
rgb3669.png
rgb3670.png
rgb3671.png
rgb3672.png
rgb3673.png
rgb3674.png
rgb3675.png
rgb3676.png
rgb3677.png
rgb3678.png
rgb3679.png
rgb3680.png
rgb3681.png
rgb3682.png
rgb3683.png
rgb3684.png
rgb3685.png
rgb3686.png
rgb3687.png
rgb3688.png
rgb3689.png
rgb3690.png
rgb3691.png
rgb3692.png
rgb3693.png
rgb3694.png
rgb3695.png
rgb3696.png
rgb3697.png
rgb3698.png
rgb3699.png
rgb3700.png
rgb3701.png
rgb3702.png
rgb3

In [ ]:
from collections import OrderedDict
from operator import itemgetter 
ids_dict_descending = OrderedDict(sorted(ids_dict.items(), key = itemgetter(1), reverse = True))
ids_dict_ascending = OrderedDict(sorted(ids_dict.items(), key = itemgetter(1), reverse = False))

In [ ]:
len(ids_dict_ascending)

111

In [ ]:
#NB we should try to prune the ids that appear less than 15 frames ,i.e a second
ids_number = len(ids_dict_ascending)
for key in ids_dict_ascending:
    value = ids_dict_ascending[key]
    if value < 30:
        ids_number = ids_number - 1
print(ids_number)     

62


In [ ]:
ids_dict_ascending

OrderedDict([(21, 1),
             (18, 1),
             (16, 1),
             (14, 1),
             (11, 1),
             (10, 1),
             (30, 1),
             (29, 1),
             (28, 1),
             (27, 1),
             (24, 1),
             (46, 1),
             (48, 1),
             (51, 1),
             (50, 1),
             (49, 1),
             (53, 1),
             (62, 1),
             (104, 1),
             (2086, 1),
             (2085, 1),
             (2169, 1),
             (19, 2),
             (15, 2),
             (12, 2),
             (8, 2),
             (7, 2),
             (6, 2),
             (2, 2),
             (1, 2),
             (26, 2),
             (23, 2),
             (40, 2),
             (44, 2),
             (54, 2),
             (56, 2),
             (60, 2),
             (61, 2),
             (176, 2),
             (795, 2),
             (1185, 2),
             (1828, 2),
             (1851, 2),
             (1901, 2),
             (2069, 

In [ ]:
!zip -r /content/tracked_images.zip /content/tracking/save

  adding: content/tracking/save/ (stored 0%)
  adding: content/tracking/save/rgb1923_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2349_mot.png (deflated 1%)
  adding: content/tracking/save/rgb1940_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2342_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2247_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2213_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2626_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2982_mot.png (deflated 2%)
  adding: content/tracking/save/rgb2532_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2345_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2527_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2024_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2684_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2717_mot.png (deflated 1%)
  adding: content/tracking/save/rgb2146_mot.png (deflated 1%)
  adding: content/trackin